In [ ]:
!pip install cdsapi

  Preparing metadata (setup.py) ... done
  Created wheel for multiurl: filename=multiurl-0.3.3-py3-none-any.whl size=21230 sha256=ca0658df5feb8a4a3dfba1ec460b4df5f3c623fd41bcbc20399fb731ad6d120e
  Stored in directory: /root/.cache/pip/wheels/be/05/e0/65a6edb0a000498aeaefbadd80228bf5ed1bdbb82840ca1692
Successfully built multiurl


In [ ]:
from google.colab import userdata
apikey = userdata.get("CDS_TOKEN")
with open("/root/.cdsapirc", "w") as f:
    print("url: https://cds.climate.copernicus.eu/api", file=f)
    print(f"key: {apikey}", file=f)

In [ ]:
import cdsapi

dataset = "reanalysis-era5-single-levels"
request = {
    "product_type": ["reanalysis"],
    "variable": ["mean_sea_level_pressure"],
    "year": ["2020"],
    "month": [
        "01", "02", "03",
        "04", "05", "06",
        "07", "08", "09",
        "10", "11", "12"
    ],
    "day": [
        "01", "02", "03",
        "04", "05", "06",
        "07", "08", "09",
        "10", "11", "12",
        "13", "14", "15",
        "16", "17", "18",
        "19", "20", "21",
        "22", "23", "24",
        "25", "26", "27",
        "28", "29", "30",
        "31"
    ],
    "time": [
        "00:00", "01:00", "02:00",
        "03:00", "04:00", "05:00",
        "06:00", "07:00", "08:00",
        "09:00", "10:00", "11:00",
        "12:00", "13:00", "14:00",
        "15:00", "16:00", "17:00",
        "18:00", "19:00", "20:00",
        "21:00", "22:00", "23:00"
    ],
    "data_format": "netcdf",
    "download_format": "zip",
    "area": [65, -120, 0, 0]
}

client = cdsapi.Client()
client.retrieve(dataset, request, "/content/data.zip")


2024-12-06 02:33:44,025 INFO [2024-09-28T00:00:00] **Welcome to the New Climate Data Store (CDS)!** This new system is in its early days of full operations and still undergoing enhancements and fine tuning. Some disruptions are to be expected. Your 
[feedback](https://jira.ecmwf.int/plugins/servlet/desk/portal/1/create/202) is key to improve the user experience on the new CDS for the benefit of everyone. Thank you.
INFO:datapi.legacy_api_client:[2024-09-28T00:00:00] **Welcome to the New Climate Data Store (CDS)!** This new system is in its early days of full operations and still undergoing enhancements and fine tuning. Some disruptions are to be expected. Your 
[feedback](https://jira.ecmwf.int/plugins/servlet/desk/portal/1/create/202) is key to improve the user experience on the new CDS for the benefit of everyone. Thank you.
2024-12-06 02:33:44,030 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
INFO:datapi.le

cf48bdf4dd229f28543766368b03ca7b.zip:   0%|          | 0.00/1.48G [00:00<?, ?B/s]

'/content/data.nc'

In [ ]:
import requests
import os

url = "https://object-store.os-api.cci2.ecmwf.int:443/cci2-prod-cache/cf48bdf4dd229f28543766368b03ca7b.zip"
file_path = "/content/data.zip"


response = requests.get(url, stream=True)
response.raise_for_status()  # Raise an exception for bad status codes

with open(file_path, "wb") as f:
  for chunk in response.iter_content(chunk_size=8192): #8KB chunks
    f.write(chunk)

print(f"File downloaded to: {file_path}")

File downloaded to: /content/data.zip


In [ ]:
!unzip data.zip

Archive:  data.zip
  inflating: data_stream-oper_stepType-instant.nc  


In [2]:
import xarray as xr

filename = 'data_stream-oper_stepType-instant.nc'

# Load the NetCDF dataset
ds = xr.open_dataset(filename)
ds

<xarray.Dataset> Size: 4GB
Dimensions:     (valid_time: 8784, latitude: 261, longitude: 481)
Coordinates:
    number      int64 8B ...
  * valid_time  (valid_time) datetime64[ns] 70kB 2020-01-01 ... 2020-12-31T23...
  * latitude    (latitude) float64 2kB 65.0 64.75 64.5 64.25 ... 0.5 0.25 0.0
  * longitude   (longitude) float64 4kB -120.0 -119.8 -119.5 ... -0.5 -0.25 0.0
    expver      (valid_time) <U4 141kB ...
Data variables:
    msl         (valid_time, latitude, longitude) float32 4GB ...
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2024-12-06T02:49 GRIB to CDM+CF via cfgrib-0.9.1...

In [3]:
if 'expver' in ds:
  ds = ds.drop_vars('expver')
  print("Column 'expver' dropped successfully.")
else:
  print("Column 'expver' not found in the dataset.")

if "number" in ds.keys():
  ds = ds.drop_vars('number')
  print("Column 'number' dropped successfully.")
else:
  print("Column 'number' not found in the dataset.")

# if "valid_time" in ds.keys():
#   ds["valid_time"] = ds["valid_time"].dt.strftime("%y/%m/%d %H:%M:%S")
#   print("Column 'valid_time' convert to string successfully.")
# else:
#   print("Column 'valid_time' not found in the dataset.")

ds

Column 'expver' dropped successfully.
Column 'number' dropped successfully.


<xarray.Dataset> Size: 4GB
Dimensions:     (valid_time: 8784, latitude: 261, longitude: 481)
Coordinates:
  * valid_time  (valid_time) datetime64[ns] 70kB 2020-01-01 ... 2020-12-31T23...
  * latitude    (latitude) float64 2kB 65.0 64.75 64.5 64.25 ... 0.5 0.25 0.0
  * longitude   (longitude) float64 4kB -120.0 -119.8 -119.5 ... -0.5 -0.25 0.0
Data variables:
    msl         (valid_time, latitude, longitude) float32 4GB ...
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2024-12-06T02:49 GRIB to CDM+CF via cfgrib-0.9.1...

In [ ]:
!mkdir /content/processed

In [4]:
# Round latitude and longitude to the nearest degree
ds = ds.assign_coords(
    latitude=ds.latitude.round(0),
    longitude=ds.longitude.round(0)
)
# Group by the rounded coordinates and compute the mean temperature
ds = ds.groupby(['latitude', 'longitude']).mean()
print(ds)

<xarray.Dataset> Size: 281MB
Dimensions:     (latitude: 66, longitude: 121, valid_time: 8784)
Coordinates:
  * latitude    (latitude) float64 528B 0.0 1.0 2.0 3.0 ... 62.0 63.0 64.0 65.0
  * longitude   (longitude) float64 968B -120.0 -119.0 -118.0 ... -2.0 -1.0 -0.0
  * valid_time  (valid_time) datetime64[ns] 70kB 2020-01-01 ... 2020-12-31T23...
Data variables:
    msl         (valid_time, latitude, longitude) float32 281MB 1.011e+05 ......
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2024-12-06T02:49 GRIB to CDM+CF via cfgrib-0.9.1...


In [5]:
df = ds.to_dataframe()
df = df.reset_index()
df

,latitude,longitude,valid_time,msl
0,0.0,-120.0,2020-01-01 00:00:00,101127.507812
1,0.0,-120.0,2020-01-01 01:00:00,101140.867188
2,0.0,-120.0,2020-01-01 02:00:00,101239.109375
3,0.0,-120.0,2020-01-01 03:00:00,101312.125000
4,0.0,-120.0,2020-01-01 04:00:00,101376.914062
...,...,...,...,...
70149019,65.0,-0.0,2020-12-31 19:00:00,101094.960938
70149020,65.0,-0.0,2020-12-31 20:00:00,101125.523438
70149021,65.0,-0.0,2020-12-31 21:00:00,101170.039062
70149022,65.0,-0.0,2020-12-31 22:00:00,101164.898438


In [6]:
# Round the 'msl' column to the nearest whole number
df['msl'] = df['msl'].round(0)
df

,latitude,longitude,valid_time,msl
0,0.0,-120.0,2020-01-01 00:00:00,101128.0
1,0.0,-120.0,2020-01-01 01:00:00,101141.0
2,0.0,-120.0,2020-01-01 02:00:00,101239.0
3,0.0,-120.0,2020-01-01 03:00:00,101312.0
4,0.0,-120.0,2020-01-01 04:00:00,101377.0
...,...,...,...,...
70149019,65.0,-0.0,2020-12-31 19:00:00,101095.0
70149020,65.0,-0.0,2020-12-31 20:00:00,101126.0
70149021,65.0,-0.0,2020-12-31 21:00:00,101170.0
70149022,65.0,-0.0,2020-12-31 22:00:00,101165.0


In [7]:
import pandas as pd

# Assuming 'df' is your DataFrame with the 'valid_time' column
# and it's already in datetime format (if not, convert it first)

# Convert 'valid_time' to datetime objects if it's not already
df['valid_time'] = pd.to_datetime(df['valid_time'])

# Reference datetime (January 1, 2020, 00:00:00)
reference_datetime = pd.to_datetime('2020-01-01 00:00:00')

# Calculate the difference in hours
df['valid_time'] = (df['valid_time'] - reference_datetime).dt.total_seconds() / 3600

df

,latitude,longitude,valid_time,msl
0,0.0,-120.0,0.0,101128.0
1,0.0,-120.0,1.0,101141.0
2,0.0,-120.0,2.0,101239.0
3,0.0,-120.0,3.0,101312.0
4,0.0,-120.0,4.0,101377.0
...,...,...,...,...
70149019,65.0,-0.0,8779.0,101095.0
70149020,65.0,-0.0,8780.0,101126.0
70149021,65.0,-0.0,8781.0,101170.0
70149022,65.0,-0.0,8782.0,101165.0


In [8]:
import pandas as pd

# Assuming 'df' is your DataFrame
# Convert all columns to int8, handling potential errors
for col in df.columns:
    try:
        df[col] = df[col].astype('int32')
    except (ValueError, TypeError):
        print(f"Could not convert column '{col}' to int8. Check data types.")
        # # Handle the error as needed, e.g., convert to a different type or skip
        # # Example: Convert to int32 if int8 fails
        # try:
        #     df[col] = df[col].astype('int32')
        # except (ValueError, TypeError):
        #   print(f"Could not convert column '{col}' to int32 either. Skipping conversion for this column.")
        #   pass # Or choose another alternative for handling errors.

df

,latitude,longitude,valid_time,msl
0,0,-120,0,101128
1,0,-120,1,101141
2,0,-120,2,101239
3,0,-120,3,101312
4,0,-120,4,101377
...,...,...,...,...
70149019,65,0,8779,101095
70149020,65,0,8780,101126
70149021,65,0,8781,101170
70149022,65,0,8782,101165


In [ ]:
df.to_csv("/content/processed/msl-simpletime.csv", index=False)

In [9]:
!pip install supabase

In [10]:
import os
from supabase import create_client, Client

table_name = "msl"

from google.colab import userdata
url = userdata.get("SUPABASE_URL")
key = userdata.get("SUPABASE_KEY")

supabase: Client = create_client(url, key)

In [ ]:
import pandas as pd
from supabase import create_client, Client

chunk_size = 1000
num_chunks = (len(df) + chunk_size - 1) // chunk_size
table_name = "msl-v2"

for i in range(num_chunks):
    start = i * chunk_size
    end = min((i + 1) * chunk_size, len(df))
    chunk = df.iloc[start:end].to_dict(orient="records")
    try:
        response = supabase.table(table_name).insert(chunk).execute()
        print(f"Successfully inserted chunk {i+1} of {num_chunks}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

In [ ]:
!zip -r processed.zip /content/processed/

  adding: content/processed/chunk_0.csv (deflated 87%)


In [ ]:
!ls processed -lah

total 6.8G
drwxr-xr-x 3 root root 4.0K Dec  6 20:01 .
drwxr-xr-x 1 root root 4.0K Dec  6 19:24 ..
drwxr-xr-x 2 root root 4.0K Dec  6 20:01 .ipynb_checkpoints
-rw-r--r-- 1 root root 2.6G Dec  6 19:39 msl.csv
-rw-r--r-- 1 root root 2.5G Dec  6 19:53 msl-rounded-i-think.csv
-rw-r--r-- 1 root root 1.8G Dec  6 20:13 msl-simpletime.csv
